# Basic Plotting

For the atmospheric data, we will look at common variables in the atmospheric diagnostics. Some of the plotting in these examples are based on the <a href="https://github.com/NCAR/ADF">AMWG diagnostic framework (ADF)</a> and some are natively from the `xarray` functionality.


let's go over 3 basic plotting examples:

Exercise 1: Global lat/lon of surface temperature

Exercise 2: Zonal mean of short wave cloud forcing

Exercise 3: Temperature zonal mean with vertical levels

`xarray` will be used for the data I/O, analysis, and some plotting, `matplotlib` and `cartopy` will aid in plotting, and `numpy` for calculations

In [ ]:
import os

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cftime
import matplotlib as mpl
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
from mpl_toolkits.axes_grid1 import make_axes_locatable

The first step is to grab an atmosphere (CAM) history file from your CESM model run

In [ ]:
# Set your username here:
username = "PUT_USER_NAME_HERE"

# Here we point to the archive directory from your b.day2.1 simulation
monthly_output_path = f"/glade/scratch/{username}/archive/b.day2.1/atm/hist"

# If you were unable to successfully run the b.day2.1 simulation, then feel free to use
# this provided simulation data instead:
#monthly_output_path = "/glade/p/cesm/tutorial/tutorial_2023_archive/b.day2.1/atm/hist"

# Name of history file to plot
file_name = "b.day2.1.cam.h0.0003-07.nc"

files = os.path.join(monthly_output_path, file_name)
files

In [ ]:
ds = xr.open_dataset(files)
ds

## Exercise 1: Make a lat-lon plot of TS

To highlight plotting the variables from the CESM atmosphere (CAM) file, the first example will plot a simple global lat/lon plot of surface temperature `TS`

### Grab data from first time stamp

NOTE: This dataset has <em>only</em> one time

In [ ]:
ts_0 = ds.TS.sel({"time": ds.TS.time.values[0]}).squeeze()
ts_0

Next is to set up the map. Since we are plotting a global lat/lon, we will use the Plate Carree projection. 

In [ ]:
# define the colormap
# cmap = plt.cm.get_cmap('jet')
cmap = mpl.colormaps["jet"]

# set up the figure with a Plate Carree projection
fig = plt.figure(figsize=(15, 10))

ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Plot the first timeslice of TS
img = ax.pcolormesh(ds.lon, ds.lat, ts_0, cmap=cmap, transform=ccrs.PlateCarree())

plt.title("Surface Temperature", fontsize=20)

# Set up colorbar
plt.colorbar(img, orientation="vertical", fraction=0.0242, pad=0.01)
plt.show()

#### Question:

What code would you use in the `pcolormesh` function to set the plotting limits?

#### Question:

How could we change the central longitude?

#DROPDOWN ?

```python
ax = fig.add_subplot(1,1,1, projection=ccrs.PlateCarree(central_longitude=0))
```

A second quick example is for `xarray`'s built-in plotting which uses the `matplotlib` and `cartopy` in the background. `xarray` makes a basic plot fairly simple.

In [ ]:
# Xarray native plotting

# Set up figure and axis
fig, ax = plt.subplots(1, figsize=(20, 10))

# Plot the data straight from the xarray dataset
ts_0.plot.contourf(cmap="jet", levels=np.arange(220, 321, 5))

plt.show()

---

## Exercise 2: Zonal plot of SWCF

The second example will plot the short wave cloud forcing `SWCF` zonally.

Grab the variable data and mean over the the lon value

In [ ]:
ds_swcf = ds.SWCF

# Get all the dataset dimensions
d = ds_swcf.dims

# Grab all dimensions to mean the lon values from
davgovr = [dim for dim in d if dim not in ("lev", "lat")]

# Make new dataset of zonal mean values
ds_swcf_zonal = ds_swcf.mean(dim=davgovr)

# print some values of new zonally-averaged SWCF variable
ds_swcf_zonal

In [ ]:
# Create figure and axis
fig, ax = plt.subplots(1, figsize=(12, 7))

# Set Main title for subplots:
plt.title("Short Wave Cloud Forcing", fontsize=20)

# Plot value on y-axis and latitude on the x-axis
ax.plot(ds_swcf_zonal.lat, ds_swcf_zonal, c="#1f77b4")

ax.set_xlim(
    [max([ds_swcf_zonal.lat.min(), -90.0]), min([ds_swcf_zonal.lat.max(), 90.0])]
)

ax.set_xlabel("Latitude")
plt.show()

### Question

What code could you add to set legend for plot line?

#DROPDOWN ?

```python
label = ds_swcf.time.values[0].strftime() # -> '0001-02-01 00:00:00'
line = Line2D([0], [0], label=label,
                        color="#1f77b4")
                        
fig.legend(handles=[line],bbox_to_anchor=(-0.15, 0.15, 1.05, .102),loc="right",
                   borderaxespad=0.0,fontsize=16,frameon=False)
```

What else could go for the line legend, if anything?
#DROPDOWN ?
-> suimulation run name, season, etc.

---

## Exercise 3: Plot of zonal T

This example will plot the 3D zonal mean of temperature `T` with the pressure as the y-variable and latitude as the x-variable


```{warning}
QUESTION FOR REVIEWERS: Here are four examples of how to plot the data. I think each is valuable for plotting pressure on the y-axis and changing the y-tick scale (ie log) to get more information out of the data. 

I've added two different ways of plotting: native xarray and what the ADF uses. 

Let me know about any feedback on these, I'm sure these could be turned into exercises?

```

In [ ]:
# Remove all dimensions of length one
ds_t = ds.T.squeeze()
ds_t

#### Natively via xarray

 - y-axis is <strong>increasing</strong> with height
 - y-axis is <strong>not</strong> in log pressure

In [ ]:
# Average over all dimensions except `lev` and `lat`.
d = ds_t.dims
davgovr = [dim for dim in d if dim not in ("lev", "lat")]

DS = ds_t.mean(dim=davgovr)
DS.transpose("lat", "lev", ...)
fig, ax = plt.subplots(1, figsize=(20, 10))
DS.plot.contourf(ax=ax, y="lev", cmap="Spectral_r")
plt.show()

#### ADF style quick plot

 - y-axis is <strong>increasing</strong> with height
 - y-axis is <strong>not</strong> in log pressure

In [ ]:
# Average over all dimensions except `lev` and `lat`.
d = ds_t.dims
davgovr = [dim for dim in d if dim not in ("lev", "lat")]

DS = ds_t.mean(dim=davgovr)

lev = DS["lev"]
lat = DS["lat"]
mlev, mlat = np.meshgrid(lev, lat)

# Generate zonal plot:
fig, ax = plt.subplots(1, figsize=(15, 7))

# Create zonal plot with vertical levels
img = ax.contourf(mlat, mlev, DS.transpose("lat", "lev"), cmap="Spectral_r")

# Format axis and ticks
ax.set_xlabel("Latitude")
fig.colorbar(img, ax=ax, location="right")

#### Same as above, but:

 - y-axis is <strong>decreasing</strong> with height
 - y-axis is <strong>not</strong> in log pressure

In [ ]:
# Average over all dimensions except `lev` and `lat`.
d = ds_t.dims
davgovr = [dim for dim in d if dim not in ("lev", "lat")]

DS = ds_t.mean(dim=davgovr)

# print(DS.lev.min(),DS.lev.max())

lev = DS["lev"]
lat = DS["lat"]
mlev, mlat = np.meshgrid(lev, lat)

# Generate zonal plot:
fig, ax = plt.subplots(1, figsize=(15, 7))

# Create zonal plot with vertical levels
img = ax.contourf(mlat, mlev, DS.transpose("lat", "lev"), cmap="Spectral_r")

# Format axis and ticks
plt.gca().invert_yaxis()
ax.tick_params(which="minor", length=4, color="r")

# Set up colorbar
cbar_ax = fig.add_axes([0, 0, 0.1, 0.1])
posn = ax.get_position()

# Set position and size of colorbar position: [left, bottom, width, height]
cbar_ax.set_position([posn.x0 + posn.width + 0.005, posn.y0, 0.02, posn.height])

ax.set_xlabel("Latitude")
fig.colorbar(img, cax=cbar_ax, orientation="vertical")

#### Same as above, but:

 - y-axis is <strong>decreasing</strong> with height
 - y-axis <strong>is</strong> in log pressure

In [ ]:
# Average over all dimensions except `lev` and `lat`.
d = ds_t.dims
davgovr = [dim for dim in d if dim not in ("lev", "lat")]

DS = ds_t.mean(dim=davgovr)

# print(DS.lev.min(),DS.lev.max())

lev = DS["lev"]
lat = DS["lat"]
mlev, mlat = np.meshgrid(lev, lat)

# Generate zonal plot:
fig, ax = plt.subplots(1, figsize=(15, 7))

# Create zonal plot with vertical levels
img = ax.contourf(mlat, mlev, DS.transpose("lat", "lev"), cmap="Spectral_r")

# Format axis and ticks
plt.yscale("log")
plt.gca().invert_yaxis()
ax.tick_params(which="minor", length=4, color="r")

# Set up colorbar
cbar_ax = fig.add_axes([0, 0, 0.1, 0.1])
posn = ax.get_position()

# Set position and size of colorbar position: [left, bottom, width, height]
cbar_ax.set_position([posn.x0 + posn.width + 0.005, posn.y0, 0.02, posn.height])

ax.set_xlabel("Latitude")
fig.colorbar(img, cax=cbar_ax, orientation="vertical")